# Neo4j와 LangChain을 활용한 영화 추천 시스템

---

## 1. Neo4J DB 환경 설정

In [1]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv(override=True)

True

In [2]:
from langchain_neo4j import Neo4jGraph

# LangChain 도구 활용 - DB 연결 객체 초기화 
graph = Neo4jGraph( 
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
    database=os.getenv("NEO4J_DATABASE")
)

In [3]:
# 테스트 쿼리 실행 
cypher_query = """
MATCH (n:Movie)
RETURN COUNT(n) AS Movie_Count
"""

graph.query(cypher_query)

[{'Movie_Count': 4803}]

---

## 2. **Movie 노드** : 임베딩 필드 추가 및 벡터 인덱스 생성

- **Movie 노드**에 텍스트 데이터를 벡터화한 **임베딩 필드**를 추가함
- 영화 제목, 줄거리 등의 텍스트 정보를 **고차원 벡터**로 변환하여 저장함
- 벡터화된 데이터를 효율적으로 검색하기 위한 **벡터 인덱스**를 생성함
- 인덱스 생성 시 **벡터 차원**, **유사도 계산 방식**, **거리 함수** 등을 지정함
- 임베딩과 벡터 인덱스를 통해 **의미적 검색**의 기반을 구축함

### 2.1 임베딩 모델 초기화

- **OpenAI 임베딩 모델**을 활용하여 텍스트를 벡터로 변환하는 환경을 설정함
- 임베딩 모델은 텍스트의 **의미적 특성**을 수치화된 벡터로 표현함

In [4]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

### 2.2 벡터 인덱스 생성

- Neo4j에서 **벡터 인덱스**를 생성하여 영화 줄거리의 임베딩 벡터를 효율적으로 검색할 수 있게 함
- **movie_content_embeddings**라는 이름의 벡터 인덱스를 Movie 노드의 **content_embedding** 필드에 적용함 (필드를 새로 추가)
- 벡터 차원을 **1536차원**으로 설정하여 OpenAI의 text-embedding-3-small 모델과 호환되도록 함
- 유사도 계산 방식으로 **코사인 유사도**를 선택하여 벡터 간 각도 기반의 의미적 유사성을 측정함
- 쿼리에 **IF NOT EXISTS** 조건을 포함하여 중복 생성을 방지함

In [5]:
# 벡터 인덱스 생성
create_vector_index_query = """
// 영화 콘텐츠 임베딩을 위한 벡터 인덱스 생성
// IF NOT EXISTS: 이미 존재하는 경우 중복 생성 방지
CREATE VECTOR INDEX movie_content_embeddings IF NOT EXISTS 

// Movie 노드의 content_embedding 속성에 인덱스 적용
FOR (m:Movie) ON m.content_embedding 

// 벡터 인덱스 설정 옵션
OPTIONS {
  indexConfig: {
    `vector.dimensions`: 1536,
    `vector.similarity_function`: 'cosine'
  }
}
"""
graph.query(create_vector_index_query)

[]

In [6]:
# 벡터 인덱스 확인
check_vector_index_query = """
SHOW VECTOR INDEXES
"""
vector_indexes = graph.query(check_vector_index_query)
for index in vector_indexes:
    # 벡터 인덱스 정보 출력
    print(f"Index Name: {index['name']}")
    print(f"Type: {index['type']}")    
    print(f"Property Key: {index['properties']}")
    print("-" * 40)

Index Name: movie_content_embeddings
Type: VECTOR
Property Key: ['content_embedding']
----------------------------------------


### 2.3 임베딩 생성 및 저장

- 각 영화 제목, 태그라인, 개요에 대해 **OpenAI 임베딩**을 생성하는 과정 수행
- 빈 문자열인 경우 처리를 **건너뛰는** 예외 처리 포함
- 생성된 임베딩을 `db.create.setNodeVectorProperty` 프로시저를 통해 **content_embedding** 속성으로 저장

In [7]:
# 영화 제목과 줄거리 가져오기
movies_query = """
MATCH (m:Movie)
WHERE m.title IS NOT NULL
RETURN m.id AS id, m.title AS title, m.overview AS overview, m.tagline AS tagline
"""
movies = graph.query(movies_query)

# 배치 크기 설정
BATCH_SIZE = 100

# 임베딩 생성 및 저장 (배치 처리)
for i in range(0, len(movies), BATCH_SIZE):
    batch = movies[i:i+BATCH_SIZE]
    batch_texts = []
    batch_ids = []
    
    # 배치 데이터 준비
    for movie in batch:
        # overview와 tagline을 "\n\n"으로 결합
        content_text = f"{movie['title']}"
        if movie['tagline']:
            content_text += f"\n\n{movie['tagline']}"
        if movie['overview']:
            content_text += f"\n\n{movie['overview']}"
        
        if content_text.strip():  # 빈 문자열 확인
            batch_texts.append(content_text)
            batch_ids.append(movie['id'])
    
    try:
        if batch_texts:
            # 배치 단위로 OpenAI 임베딩 생성
            batch_embeddings = embeddings.embed_documents(batch_texts)
            
            # UNWIND를 사용한 배치 업데이트
            batch_data = [{"id": article_id, "embedding": embedding_vector} 
                         for article_id, embedding_vector in zip(batch_ids, batch_embeddings)]
            
            batch_update_query = """
            // UNWIND를 사용하여 배치 데이터를 개별 행으로 변환
            UNWIND $batch AS item

            // 영화 ID로 해당 Movie 노드 찾기
            MATCH (m:Movie {id: item.id})

            // db.create.setNodeVectorProperty 프로시저를 호출하여 벡터 속성 설정
            // 첫 번째 인자: 대상 노드, 두 번째 인자: 속성 이름, 세 번째 인자: 벡터 값
            CALL db.create.setNodeVectorProperty(m, 'content_embedding', item.embedding)

            // 업데이트된 노드 수 반환
            RETURN count(m) as updated
            """
            
            result = graph.query(batch_update_query, params={"batch": batch_data})
            print(f"배치 처리 완료: {i+1}~{min(i+len(batch_texts), len(movies))} / {len(movies)}, 업데이트됨: {result[0]['updated']}")
            
    except Exception as e:
        print(f"배치 임베딩 생성 실패 (배치 인덱스 {i}): {str(e)}")

print(f"영화 임베딩 업데이트 완료!! 총 {len(movies)}개 처리")

배치 처리 완료: 1~100 / 4803, 업데이트됨: 100
배치 처리 완료: 101~200 / 4803, 업데이트됨: 100
배치 처리 완료: 201~300 / 4803, 업데이트됨: 100
배치 처리 완료: 301~400 / 4803, 업데이트됨: 100
배치 처리 완료: 401~500 / 4803, 업데이트됨: 100
배치 처리 완료: 501~600 / 4803, 업데이트됨: 100
배치 처리 완료: 601~700 / 4803, 업데이트됨: 100
배치 처리 완료: 701~800 / 4803, 업데이트됨: 100
배치 처리 완료: 801~900 / 4803, 업데이트됨: 100
배치 처리 완료: 901~1000 / 4803, 업데이트됨: 100
배치 처리 완료: 1001~1100 / 4803, 업데이트됨: 100
배치 처리 완료: 1101~1200 / 4803, 업데이트됨: 100
배치 처리 완료: 1201~1300 / 4803, 업데이트됨: 100
배치 처리 완료: 1301~1400 / 4803, 업데이트됨: 100
배치 처리 완료: 1401~1500 / 4803, 업데이트됨: 100
배치 처리 완료: 1501~1600 / 4803, 업데이트됨: 100
배치 처리 완료: 1601~1700 / 4803, 업데이트됨: 100
배치 처리 완료: 1701~1800 / 4803, 업데이트됨: 100
배치 처리 완료: 1801~1900 / 4803, 업데이트됨: 100
배치 처리 완료: 1901~2000 / 4803, 업데이트됨: 100
배치 처리 완료: 2001~2100 / 4803, 업데이트됨: 100
배치 처리 완료: 2101~2200 / 4803, 업데이트됨: 100
배치 처리 완료: 2201~2300 / 4803, 업데이트됨: 100
배치 처리 완료: 2301~2400 / 4803, 업데이트됨: 100
배치 처리 완료: 2401~2500 / 4803, 업데이트됨: 100
배치 처리 완료: 2501~2600 / 4803, 업데이트됨: 100
배치 처리 완